# Generate the trainning and validation set file 
1. list txt for training and validation
2. h5 file for training and validation
3. txt file refer to h5 file for training and validation

In [2]:
import h5py
import numpy as np
#from scipy import stats
#from random import shuffle
import os
os.chdir('../../')
import sys
import time
sys.path.insert(0, './python')
sys.path.append('/local-scratch/xca64/tmp/caffe-master/python/myFunc')
import caffe
import numpy as np
from pylab import *
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tempfile
from colorConstancy import * 

caffe.set_device(0)
caffe.set_mode_gpu()

In [3]:
def run_solver(solver, niter, disp_interval, test_interval):
    blobs = ('loss', 'acc')
    loss, acc = (np.zeros(niter), np.zeros(niter))
    test_loss = (np.zeros(niter/test_interval)) 
    for it in range(niter):
        solver.step(1)  # run a single SGD step in Caffe
        loss[it] = (solver.net.blobs['loss'].data.copy())
        acc[it] = 0#(solver.net.blobs['loss_ang'].data.copy())
        if it % disp_interval == 0 or it + 1 == niter:
            loss_disp = 'loss: %.3f'%loss[it]
            print '%3d) %s Angular Erro %.3f' % (it, loss_disp, acc[it])     
            #print(solver.net.blobs['fc8_flickr'].data[1], solver.net.blobs['illu'].data[1])
        # if it % test_interval == 0:
        #     print "%3d) test loss is %.3f"% (it ,solver.test_nets[0].blobs['accuracy'].data.copy())
        #     test_loss[it//test_interval] = solver.test_nets[0].blobs['accuracy'].data.copy()
    # Save the learned weights from both nets.
    weight_dir = tempfile.mkdtemp()
    name = 'firstTry'
    weights = {}
    filename = 'weights.%s.caffemodel' % name
    
    weights[name] = os.path.join(weight_dir, filename)
    solver.net.save(weights[name])
    
    return loss, test_loss, acc, weights

#Generate all training and validation files

In [19]:
fname = "/home/xca64/remote/GitHub/colorP/dataSet/geler/Gehler_Extras/Illuminants.txt"
with open(fname) as f:
    content = f.readlines()

illu = np.zeros([len(content)/3,3]) #  len(content_src)
src = []
for i in range(len(content)/3): #  len(content_src)
    illu[i, :] = np.array([float(x) for x in ((content[i*3+1][:-3]).split(','))])
    src.append("/local-scratch/xca64/geler/source/" + content[i*3][:-2] + " " + str(i)+"\n")


#crossvalidation 1 1 '1' : 
# seprater = 2* len(content)/9 #len(content_src)/3 
# illu_train = illu[:seprater,:]
# illu_test = illu[seprater:,:]
# src_train = src[:seprater]
# src_test = src[seprater:]

#crossvalidation 1 '1' 1 : 
seprater1 = len(content)/9 #len(content_src)/3 
seprater2 = 2* len(content)/9 #len(content_src)/3 
illu_train = np.vstack((illu[:seprater1,:],illu[seprater2:,:] ))
illu_test = illu[seprater1:seprater2,:]
src_train = src[:seprater1] + src[seprater2:]
src_test = src[seprater1:seprater2]

#crossvalidation '1' 1 1 : 
# seprater = len(content)/9 #len(content_src)/3 
# illu_train = illu[seprater:,:]
# illu_test = illu[:seprater,:]
# src_train = src[seprater:]
# src_test = src[:seprater]

# print np.shape(illu_train)
# print np.shape(illu_test)
# print len(src_train)
# print len(src_test)


In [20]:
dataPath = "/home/xca64/remote/data/"
fileName = "test_ill.h5"
rmCommand = "rm -rf " +  dataPath + fileName 
os.system(rmCommand)
fileName = "train_ill.h5"
rmCommand = "rm -rf " +  dataPath + fileName 
os.system(rmCommand)

f = open('/home/xca64/remote/data/train_src.txt', 'w')
for i in range(len(src_train)):
    f.write(src_train[i])
f.close()
ft = open('/home/xca64/remote/data/test_src.txt', 'w')
for i in range(len(src_test)):
    ft.write(src_test[i])
ft.close()

with h5py.File('/home/xca64/remote/data/test_ill.h5') as h:
    h['illu'] = illu_test
with h5py.File('/home/xca64/remote/data/train_ill.h5') as h:
    h['illu'] = illu_train

#Define Training Solver and Run

In [4]:
#Training Solver
solver = caffe.SGDSolver('models/color_constancy/gehler_482_solver.prototxt')
solver.net.copy_from('models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel')

In [5]:
niters = 1000
loss_1, test_loss, acc_1, weights_1 = run_solver(solver, niters,20, 5000)
solver.net.save('models/color_constancy/result/gehler_482_3000_iters_cross_validation.caffemodel')
np.save('/home/xca64/remote/data/geler_crossvalidation_1.npy', loss_1) 

  0) loss: 111.012 Angular Erro 0.000
 20) loss: 5.711 Angular Erro 0.000
 40) loss: 3.942 Angular Erro 0.000
 60) loss: 5.191 Angular Erro 0.000
 80) loss: 5.255 Angular Erro 0.000
100) loss: 4.310 Angular Erro 0.000
120) loss: 5.565 Angular Erro 0.000
140) loss: 9.569 Angular Erro 0.000
160) loss: 7.118 Angular Erro 0.000
180) loss: 3.027 Angular Erro 0.000
200) loss: 2.894 Angular Erro 0.000
220) loss: 3.565 Angular Erro 0.000
240) loss: 4.528 Angular Erro 0.000
260) loss: 2.814 Angular Erro 0.000
280) loss: 3.700 Angular Erro 0.000
300) loss: 7.047 Angular Erro 0.000
320) loss: 8.490 Angular Erro 0.000
340) loss: 5.053 Angular Erro 0.000
360) loss: 2.650 Angular Erro 0.000
380) loss: 2.968 Angular Erro 0.000
400) loss: 3.135 Angular Erro 0.000
420) loss: 3.625 Angular Erro 0.000
440) loss: 2.762 Angular Erro 0.000
460) loss: 2.852 Angular Erro 0.000
480) loss: 6.083 Angular Erro 0.000
500) loss: 4.350 Angular Erro 0.000
520) loss: 2.654 Angular Erro 0.000
540) loss: 2.888 Angular E

#Define Validation Solver and Run

In [6]:
solver = caffe.SGDSolver('models/color_constancy/gehler_482_solver_validation.prototxt')
solver.net.copy_from('models/color_constancy/result/gehler_482_3000_iters_cross_validation.caffemodel')

In [7]:
validationIters = 100 # 500 #every time forward a batch size image
validationSetSize = 161 #number of validation images
allAngulareErr = np.array([])
allGroundTruth = np.array([0,0,0])
allImg = np.zeros([1,3,227,227])
for it in range(validationIters):
    solver.net.forward()
    estimatedIllu = solver.net.blobs['fc8_flickr'].data.copy()  #batch size illumination estimation result
    groundtruethIllu = solver.net.blobs['illu'].data.copy()  #batch size illumination estimation result
    #print solver.net.blobs['label'].data.copy() 
    #allGroundTruth = np.vstack((allGroundTruth, estimatedIllu))
    #allImg = np.vstack((allImg, solver.net.blobs['data'].data.copy() ))
    allAngulareErr = np.hstack((allAngulareErr, np.squeeze( multiangle(estimatedIllu, groundtruethIllu))))
    #print np.shape(allAngulareErr)
    #allAngulareErr = np.vstack((allAngulareErr, multiangle(estimatedIllu, groundtruethIllu)))

In [ ]:
for i in range(len(allAngulareErr)):
    for j in range(len(allAngulareErr)):
        if allAngulareErr[i] == allAngulareErr[j] and i != j:
            print i
print "Done"

# Save the result

In [8]:
final_result =[];
final_result.append(np.mean(allAngulareErr))
final_result.append(np.median(allAngulareErr))
final_result.append((np.percentile(allAngulareErr,25)+ np.percentile(allAngulareErr,75) + 2 *np.median(allAngulareErr))/4)
final_result.append(np.amin(allAngulareErr))
final_result.append(np.percentile(allAngulareErr,95))
print final_result
# fileName = "final_result.txt"
# rmCommand = "rm -rf /home/xca64/remote/data/" + fileName
# #os.system(rmCommand)
# f = open('/home/xca64/remote/data/final_result.txt', 'a')
# f.write("result for geler crossvalidation '1' 1 1 :\n")
# f.write(str(final_result))
# f.write("\n")
# f.close()

[4.9682476252972672, 3.8265616157506339, 4.1323755451534332, 0.044234659612636382, 12.550987553473167]


In [ ]:
print np.mean(allAngulareErr)

#Only for debug

In [ ]:
solver.net.forward()

In [ ]:
img = solver.net.blobs['data'].data.copy()

In [ ]:
allImg = np.zeros([1,3,227,227])

In [ ]:
image = allImg[162,...]
plt.imshow(deprocess_net_image(image))

In [ ]:
# Helper function for deprocessing preprocessed images, e.g., for display.
def deprocess_net_image(image):
    image = image.copy()              # don't modify destructively
    image = image[::-1]               # BGR -> RGB
    image = image.transpose(1, 2, 0)  # CHW -> HWC
    image += [123, 117, 104]          # (approximately) undo mean subtraction

    # clamp values in [0, 255]
    image[image < 0], image[image > 255] = 0, 255

    # round and cast from float32 to uint8
    image = np.round(image)
    image = np.require(image, dtype=np.uint8)

    return image